In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2

# Movimiento armónico simple y oscilador amortiguado

In [2]:
def Minimos_cuadrados(x,y,n=1):
    l = x.shape[0]
    b = y
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    AT = np.dot(A.T,A)
    bT = np.dot(A.T,b)

    xsol = np.linalg.solve(AT,bT)
    
    return xsol

def GetModel(x,p):
    
    y = 0.
    for i in range(len(p)):
        y += p[i]*x**i
        
    return y

def GetError(x,y,p,n=1):
    
    l = x.shape[0]
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    # Residuos
    R = y - np.dot(A,p)
    
    sigma2 = np.dot(R.T,R)/(len(y)-len(p))
    
    Cov = sigma2*np.linalg.inv(np.dot(A.T,A))
    
    R2 = None
    if np.shape(Cov) == (2,2):
        R2 = (Cov[0][1]**2)/(Cov[0][0]*Cov[1][1])

    return (Cov,R2,R)

def Get_sigma(Cov):
    sigmaai = np.sqrt(np.diagonal(Cov))
    return sigmaai[::-1]

def Error_porcentual(X_real,X_exp):
    return (np.abs(X_exp - X_real)/X_real)

X = sym.Symbol('x',real=True)
Y = sym.Symbol('y',real=True)

# Datos

In [3]:
h = np.array([1.5, 1.35, 1.2, 1.05, 0.9, 0.75]) #m

#Cobre
m_cobre = 0.05 #Kg
T = np.array([2.47, 2.36, 2.21, 2.04, 1.91, 1.74]) #s

#Icopor
m_icopor = 0.0414 #Kg
g = np.array([0.012, 0.013, 0.01, 0.009, 0.01, 0.01]) #?
w = np.array([2.478, 2.611, 2.774, 2.971, 3.172, 3.467]) #?

# Análisis